### Experimento básico de fine-tuning do LLaVa

In [12]:
# type: ignore
from os.path import join

KAGGLE_CREDENTIALS_PATH = join('..' ,'.secrets', 'kaggle.json')
DATA_PATH = join('..', '.data')

In [13]:
# type: ignore
from os import environ
from json import load
from zipfile import ZipFile

import pandas as pd

with open(KAGGLE_CREDENTIALS_PATH, 'r', encoding='utf-8') as file:
    json = load(file)
    environ['KAGGLE_USERNAME'] = json['username']
    environ['KAGGLE_KEY'] = json['key']

import kaggle

kaggle.api.authenticate()
kaggle.api.competition_download_file('siim-isic-melanoma-classification',
                                     'train.csv',
                                     path=DATA_PATH,
                                     quiet=False,
                                     force=False)

with ZipFile(join(DATA_PATH, 'train.csv.zip'), 'r') as file:
    file.extractall(join(DATA_PATH, 'extracted'))

training_data = pd.read_csv(join(DATA_PATH, 'extracted', 'train.csv'))

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
